In [4]:
import os
import pandas as pd
from tabulate import tabulate
import ftplib
import time
from forex_python.converter import CurrencyRates
from configparser import ConfigParser


parser = ConfigParser()
_ = parser.read('notebook.cfg')

HURT_HOSTNAME = parser['hurt']['HURT_HOSTNAME']
HURT_USERNAME = parser['hurt']['HURT_USERNAME']
HURT_PASSWORD = parser['hurt']['HURT_PASSWORD']

SHOP_HOSTNAME = parser['shop']['SHOP_HOSTNAME']
SHOP_USERNAME = parser['shop']['SHOP_USERNAME']
SHOP_PASSWORD = parser['shop']['SHOP_PASSWORD']
SHOP_UPLOAD_PATH = parser['shop']['SHOP_UPLOAD_PATH']

shopProductsFilename = 'product.csv'
# 0 (1) - Indeks
# 1 (2) - change ID: N = new (show), A = change (show), X = discontinued (NO show), R = remaining stock (NO show), V = temporarily delisted (NO show), W = re-listed (show)
# 6 (7) - name
# 22 (23) - order unit quantity 
# 37 (38) - price netto
# 60 (61) - item type F = freshness (with daily prices), T = dry, W = natural goods, P = deposit (this item is the deposit itself!), A = item from freshness price list (current offer)
# 65 (66) - Base price unit 
columnsToImport = [0,1,6,22,37,60,65]
columnsToImportNames = ['Indeks', 'Change', 'Name', 'Quantity', 'Price', 'ItemType', 'Unit']

priceWithMargin = 1.5
# prepareIndexesForShop()

def prepareIndexesForShop():
    shopProductsPath = os.path.join(shopProductsFilename)
    shopProducts = pd.read_csv(shopProductsPath, sep = ';', dtype={'Indeks': 'str'})

    indeks = shopProducts['Indeks']
    shopIndeksNoNull = indeks.dropna()
    shopIndeksNoNullCleared = shopIndeksNoNull.str.extract('(\d+)', expand=False)
    
    shopProductsWithClearedIndex = shopProducts[shopProducts['Indeks'].isin(shopIndeksNoNullCleared)].copy()
    shopProductsWithClearedIndex.loc[:,('Wyświetlany')]=0

def filterIndeksDigitOnly(df):
    return df[df['Indeks'].astype(str).str.isdigit()]

def filterInByIndeks(df, isInDf):
    return df[df['Indeks'].isin(isInDf)].copy()

def filterOutByIndeks(df, isInDf):
    return df[~df['Indeks'].isin(isInDf)]

def getDataFromFile(filename):
    # datesOld = getDatesFromCSV(filename)
    # print(datesOld)
    fetchFileFromHurtFtp(filename)
    # datesNew = getDatesFromCSV(filename)
    
    # print(datesNew)
    return getDfFromCSV(filename)

def getDfFromCSV(path):
    df = pd.read_csv(path, sep = ';', encoding='CP850', skiprows=[0], header=None, usecols=columnsToImport, names=columnsToImportNames, dtype={'Indeks': 'str'})
    print(f"File: {path}, original lines: {len(df.index)}")
    return df

def getDatesFromCSV(path):
    dates = pd.read_csv(filename, sep = ';', encoding='iso-8859-1', nrows=1, header=None, usecols=[7,8]).values.tolist()[0]
    print(f"File: {path}, dates: {dates}")
    return dates

def fetchFileFromHurtFtp(filename):
    ftp_server = ftplib.FTP(HURT_HOSTNAME, HURT_USERNAME, HURT_PASSWORD)
    # ftp_server.dir()
    ftp_server.encoding='CP850'

    with open(filename, "wb") as file:
        ftp_server.retrbinary(f"RETR {filename}", file.write) 
        
    ftp_server.quit()
    
def convertToFloat(str):
    return float(str.replace(',','.'))

def clearName(name):
    return name.replace('> ','')

def addTotalPriceColumn(df):
    df['TotalPrice'] = (df['Price'].apply(convertToFloat)*df['Quantity'].apply(convertToFloat)).apply(lambda x : format(x, '.2f'))

def addTotalPricePLNColumn(df):
    df['PricePLN'] = (df['Price'].apply(convertToFloat)*df['Quantity'].apply(convertToFloat)).apply(lambda x : format(x, '.2f'))
    
def addShowColumn(df):
    df['Show'] = df['Change'].isin(['N','A','W']).apply(int)

def sameDates(l1, l2):
   l1.sort()
   l2.sort()
   if(l1==l2):
      return 1
   else:
      return 0
    
def getTimestamp():
    return time.strftime("%Y%m%d-%H%M%S_")

def uploadFiletoFtp(filename):
    ftp_server = ftplib.FTP(SHOP_HOSTNAME, SHOP_USERNAME, SHOP_PASSWORD)
    ftp_server.cwd(SHOP_UPLOAD_PATH)
    file = open(filename,'rb')
    ftp_server.storbinary(f"STOR {filename}", file) 
    ftp_server.quit()

def addPricePLNColumn(df):
    curr = CurrencyRates()
    euroPlnRate = curr.get_rate('EUR', 'PLN')
    print(f"Euro price: {euroPlnRate}")
    df['PricePLN'] = (df['Price'].apply(convertToFloat)*priceWithMargin*euroPlnRate).apply(lambda x : format(x, '.2f'))


In [6]:
# Get shop product items
shopProductsPath = os.path.join(shopProductsFilename)
shopProducts = pd.read_csv(shopProductsPath, sep = ';', dtype={'Indeks': 'str'})

# print(type(shopProducts))
shopProducts = shopProducts[~shopProducts['Kategoria'].isin(['imp'])].copy()

print(f"All shop products count (NOT IN 'imp' CATEGORY): {len(shopProducts)}")
# print(shopProducts)

indeks = shopProducts['Indeks']
# print(type(indx))
# print(indeks)
shopIndeksNoNull = indeks.dropna()
# print(shopIndeksNoNull)
shopIndeksNoNullCleared = shopIndeksNoNull.str.extract('(\d+)', expand=False)
# print(shopIndeksNoNullCleared)

# print(shopProducts[shopProducts['Indeks'].isin(shopIndeksNoNullCleared)])


shopProductsWithClearedIndex = shopProducts[shopProducts['Indeks'].isin(shopIndeksNoNullCleared)].copy()
# print(type(shopProductsWithClearedIndex['Wyświetlany'][1]))
# print(shopProducts[shopProducts['Indeks'].isin(shopIndeksNoNullCleared)])

# shopProductsWithClearedIndex['Wyświetlany']=0
# shopProductsWithClearedIndex.loc[:,('Wyświetlany')]=0
# shopProductsWithClearedIndex.assign(Wyświetlany=0)
print(f"Shop products with indeks count: {len(shopProductsWithClearedIndex)}")
print(shopProductsWithClearedIndex)

# print(type(shopIndeksNoNull))
# print(shopIndeksNoNull.astype(str))
# print(indxNoNull.map('{:,.2f}'.format))
# print(tabulate(products,headers='firstrow'))

All shop products count (NOT IN 'imp' CATEGORY): 1562
Shop products with indeks count: 1104
        id                                            Obraz  \
13    9604             http://chrupnazdrowie.pl/img/p/0.jpg   
404   9213  http://chrupnazdrowie.pl/img/p/1/5/7/9/1579.jpg   
452   9165  http://chrupnazdrowie.pl/img/p/1/5/4/3/1543.jpg   
533   9084  http://chrupnazdrowie.pl/img/p/2/1/1/1/2111.jpg   
1115  8502  http://chrupnazdrowie.pl/img/p/1/8/7/1/1871.jpg   
...    ...                                              ...   
9482    93     http://chrupnazdrowie.pl/img/p/6/4/7/647.jpg   
9483    92     http://chrupnazdrowie.pl/img/p/6/4/5/645.jpg   
9488    86     http://chrupnazdrowie.pl/img/p/2/7/0/270.jpg   
9490    84     http://chrupnazdrowie.pl/img/p/6/3/1/631.jpg   
9510    45     http://chrupnazdrowie.pl/img/p/1/1/5/115.jpg   

                                                  Nazwa   Indeks  \
13                                   Stewia Tabs 300St.   719086   
404            

In [ ]:
filename = "PLF.bnn" # fruits and vegetables
vegeHurtCleared = filterIndeksDigitOnly(getDataFromFile(filename))
vegeBnnShop = filterInByIndeks(vegeHurtCleared, shopIndeksNoNullCleared)
print(f"File: {filename}, Shop lines: {len(vegeBnnShop.index)}")

filename = "PL_FRISCH.bnn" # dairy, meat, fish, cheese and other temperature-controlled goods
freshBnnShop = filterInByIndeks(filterIndeksDigitOnly(getDataFromFile(filename)), shopIndeksNoNullCleared)
print(f"File: {filename}, Shop lines: {len(freshBnnShop.index)}")

filename = "PL_FOOD.bnn" # other food (no special temperature requirements)
dryBnnShop = filterInByIndeks(filterIndeksDigitOnly(getDataFromFile(filename)), shopIndeksNoNullCleared)
print(f"File: {filename}, Shop lines: {len(dryBnnShop.index)}")

# print(dryBnn)

productsFromShopInHurt = pd.concat([vegeBnnShop, freshBnnShop, dryBnnShop])
# addTotalPriceColumn(allProducts)
addShowColumn(productsFromShopInHurt)
    
addPricePLNColumn(productsFromShopInHurt)  
shopProductsToUpdate = shopProductsWithClearedIndex.merge(productsFromShopInHurt[['Indeks', 'Show', 'PricePLN']], on=['Indeks'], how='left' )
shopProductsToUpdate['Show'] = shopProductsToUpdate['Show'].fillna(0)
shopProductsToUpdate['Show'] = shopProductsToUpdate['Show'].astype(int)

outputFilename = "update.csv"
outputFilenameWithTimestamp = getTimestamp()+outputFilename
print(f"Update filename: {outputFilenameWithTimestamp}")
pd.DataFrame.to_csv(shopProductsToUpdate, outputFilenameWithTimestamp, sep=';', na_rep='.', index=False)

uploadFiletoFtp(outputFilenameWithTimestamp)

print(shopProductsToUpdate[shopProductsToUpdate['Show'].eq(0)])
print(productsFromShopInHurt[~productsFromShopInHurt['Change'].isin(["A"])])


In [ ]:
# the idea for categories:
# take products from hurt
# for each name check each product name from shop
# if match then set the category from this shop product


# filename = "PLF.bnn" # vege and fruit
# filename = "PL_FRISCH.bnn"
filename = "PL_FOOD.bnn"

hurtIndexCleared = filterIndeksDigitOnly(getDataFromFile(filename))
# print(len(hurtIndexCleared))
# filteredInByShop = filterInByIndeks(hurtIndexCleared, shopIndeksNoNullCleared)
# print(len(filteredInByShop))


inHurtNotInShop = filterOutByIndeks(hurtIndexCleared, shopIndeksNoNullCleared)
print(f"In hurt, not in shop: {len(inHurtNotInShop)}")

addPricePLNColumn(inHurtNotInShop)


# print(filteredOutByShop['Name'].apply())

# print(inHurtNotInShop)
inHurtNotInShop['NamePL'] = inHurtNotInShop['Name'].apply(translateDEtoPL)
inHurtNotInShop['Kat'] = "imp"
inHurtNotInShop['Show'] = 0
inHurtNotInShop['Ilosc'] = 30
inHurtNotInShop['TaxID'] = 2
print(inHurtNotInShop)

# filename = "new.csv"
filenameWithTimestamp = getTimestamp()+"new_"+filename+".csv"
print(filenameWithTimestamp)
pd.DataFrame.to_csv(inHurtNotInShop, filenameWithTimestamp, sep=';', na_rep='.', index=False)
uploadFiletoFtp(filenameWithTimestamp)

In [36]:
from deep_translator import GoogleTranslator

def translateDEtoPL(word):
    translated = GoogleTranslator(source='auto', target='pl').translate(word) 
    translated = clearName(translated)
    # print(f"{word}  -->  {translated}")
    return translated

In [370]:
# import re

hurt = pd.read_csv("20211216-183138_new.csv", sep = ';', dtype={'Indeks': 'str'})

shopProductsPath = os.path.join('c:' + os.sep, 'Users', 'Mario', 'Desktop', 'chrup', 'shop', 'products.csv')
shop = pd.read_csv(shopProductsPath, sep = ';', dtype={'Indeks': 'str'})
# print(hurt['NamePL'])
# print(shop[['Nazwa', 'Kategoria']])

testStr = 'Zielsona herbsata Cupser Teas 20x1,75g'
# print(testStr1.split())

testAlphaOnly = list(filter(lambda w: w.isalpha(), testStr.split()))
# print(testAlphaOnly)

# print (shop['Nazwa'].values)
# print (shop['Nazwa'][1].split())
# print(list(filter(lambda w: w.isalpha(), shop['Nazwa'].split())))

def dupa(nameShop, nameHurt):
    # print(nameHurt)
    # print(nameShop)
    
    longWordsOnly = list(filter(lambda w: w.isalpha() and len(w)>3, nameShop.split()))
    
    # if (any(elem in longWordsOnly for elem in nameHurt)):
        # print(f"hurt: {nameHurt}")
        # print(f"shop: {longWordsOnly}")
    #     print(any(elem in longWordsOnly for elem in nameShop))
    
    return any(elem in longWordsOnly for elem in nameHurt)
    
def getCategory(nameHurt):
    # print(nazwa)
    # print(nazwa.split())
    longWordsOnly = list(filter(lambda w: w.isalpha() and len(w)>3, nameHurt.split()))
    # print(f"\nhurt1: {longWordsOnly}")
    kat = ["Świeże owoce", "Świeże warzywa"]
    vegeShop = shop[shop['Kategoria'].isin(kat)]
    # print(shop['Nazwa'].apply(lambda x: dupa(x, longWordsOnly)))
    cats = vegeShop[vegeShop['Nazwa'].apply(lambda shop: dupa(shop, longWordsOnly))]['Kategoria'].tolist()
    # print(f"Kat: {cats}")
    # print(f"Kat: {type(cats)}")
    
    
    if(len(cats)==0):
        return 
    if(len(cats)==1):
        return cats[0]
    if(len(cats)>1):
        return max(set(cats), key = cats.count)
    
    # print(shop['Nazwa'].apply(lambda x: dupa(x, longWordsOnly)))
    # hurt['NamePL'].apply()
    
    # print(any(elem in nazwa.split() for elem in testAlphaOnly))
    
    # t = filter(lambda w: w.isalpha(), nazwa.split())
    # print(list(t))
    
    

hurt['KatPL'] = hurt['NamePL'].apply(getCategory)

print(hurt[['NamePL' ,'KatPL']] )


print(len(hurt[hurt['KatPL'].isnull()] ))
# str1=shop['Nazwa'][1].split()
# print(str1)
# print(any(elem in str1 for elem in testAlphaOnly))

                                   NamePL         KatPL
0                  Cytrynowy Fino Kal 2-3          None
1                 Cytrynowy Meyer Kal 3-4          None
2             Cytryna Interdonato cal 3-4  Świeże owoce
3                                 Limonki          None
4    Biały grejpfrut, położyć 12-14 sztuk          None
..                                    ...           ...
242                      Spinat, przegraj          None
243                      Okablowanie 8 St          None
244                       mięta pieprzowa          None
245             Dzikie pomidory, przegraj          None
246             Prezent / konkurs ABA-BIO          None

[247 rows x 2 columns]
167
